In [28]:
import os
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn.functional as F
import torch

from math import floor
import pandas as pd
import scipy.io as sio
import csv
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix,f1_score

import torch.nn as nn
from typing import List, Optional, Tuple, Union
import torch.distributed as dist
import torchvision.transforms as T
import pytorch_lightning as pl

In [29]:
def load_hasc_ds(path, window, mode='train'):

    X, lbl = extract_windows_hasc(path, window, 5)

    if mode == "all":
        return X, lbl

    train_size = floor(0.8 * X.shape[0])
    print(train_size)
    if mode =="train":
        trainx = X[0:train_size]
        trainlbl =lbl[0:train_size]
        idx = np.arange(trainx.shape[0])
        np.random.shuffle(idx)
        trainx = trainx[idx,]
        trainlbl = trainlbl[idx]
        print('train samples : ', train_size)
        return trainx, trainlbl

    else:
        testx = X[train_size:]
        testlbl = lbl[train_size:]
        print('test shape {} and number of change points {} '.format(testx.shape, len(np.where(testlbl>0)[0])))

        return testx, testlbl



def extract_windows_hasc(path, window_size, step):

    dataset = sio.loadmat(path+"hasc.mat")
    window_size
    windows = []
    lbl = []
    first = True
    num_cp = 0
    x = np.array(dataset['Y'])
    cp = np.array(dataset['L'])

    ts = np.sqrt(np.power(x[:, 0], 2) + np.power(x[:, 1], 2) + np.power(x[:, 2], 2))
    for i in range(0, ts.shape[0] - window_size, windows_size // 5):
        windows.append(np.array(ts[i:i + window_size]))
        is_cp = np.where(cp[i:i + window_size] == 1)[0]
        if is_cp.size == 0:
            is_cp = [0]
        else:
            num_cp += 1
        lbl.append(is_cp[0])

    print("number of samples : {} /  number of samples with change point : {}".format(len(windows), num_cp))
    windows = np.array(windows)

    return windows, np.array(lbl)

def load_usc_ds(path, window, mode='train'):
    X, lbl = extract_windows_usc(path, window, mode)

    if mode == "all":
        return X, lbl
    train_size = int(floor(0.8 * X.shape[0]))
    if mode == "train":
        trainx = X[0:train_size]
        trainlbl = lbl[0:train_size]
        idx = np.arange(trainx.shape[0])
        np.random.shuffle(idx)
        trainx = trainx[idx,]
        trainlbl = trainlbl[idx]
        print('train samples : ', train_size)
        return trainx, trainlbl

    else:
        testx = X[train_size:]
        testlbl = lbl[train_size:]
        print('test shape {} and number of change points {} '.format(testx.shape, len(np.where(testlbl > 0)[0])))

        return testx, testlbl


def extract_windows_usc(path, window_size, mode="train"):
    windows = []
    lbl = []
    dataset = sio.loadmat(path+"usc.mat")

    ts = np.array(dataset['Y'])
    ts = ts[:,0]
    cp = np.array(dataset['L'])
    cp = cp[:,0]

    num_cp = 0

    for i in range(0, ts.shape[0] - window_size, window_size // 5):
        windows.append(np.array(ts[i:i + window_size]))
        is_cp = np.where(cp[i:i + window_size] == 1)[0]
        if is_cp.size == 0:
            is_cp = [0]
        else:
            num_cp += 1
        lbl.append(is_cp[0])

    print("number of samples : {} /  number of samples with change point : {}".format(len(windows), num_cp))
    windows = np.array(windows)

    return windows, np.array(lbl)

def load_dataset(path, ds_name, win, bs, mode="train"):
    if ds_name == 'HASC':
        trainx, trainlbl = load_hasc_ds(path, window = win, mode=mode)
    elif ds_name == "USC":
        trainx, trainlbl = load_usc_ds(path, window= win, mode=mode)
    else:
        raise ValueError("Undefined Dataset.")

    trainlbl = trainlbl.reshape((trainlbl.shape[0], 1))
    print(trainx.shape, trainlbl.shape)
    dataset = np.concatenate((trainlbl, trainx), 1)

    print("dataset shape : ", dataset.shape)
    if mode == "test":
        return dataset

    train_ds = TensorDataset(torch.from_numpy(dataset))
    return train_ds

def load_future(path, ds_name, win, bs, mode="train"):
    if ds_name == 'HASC':
        trainx, trainlbl = load_hasc_ds(path, window = 2 * win, mode=mode)
    elif ds_name == "USC":
        trainx, trainlbl = load_usc_ds(path, window=2 * win, mode=mode)
    else:
        raise ValueError("Undefined Dataset.")

    trainlbl = trainlbl.reshape((trainlbl.shape[0], 1))
    print(trainx.shape, trainlbl.shape)
    dataset = np.concatenate((trainlbl, trainx), 1)

    print("dataset shape : ", dataset.shape)
    if mode == "test":
        return dataset

    train_ds = TensorDataset(torch.from_numpy(dataset))
    return train_ds

In [4]:
def estimate_CPs(sim, gt, name, train_name, metric='cosine', threshold=0.5):

    est_cp = np.zeros(sim.shape[0])
    est_cp[np.where(sim < threshold)[0]] = 1
    tn, fp, fn, tp = confusion_matrix(gt, est_cp).ravel()
    f1 = f1_score(gt, est_cp)

    gt_id = np.where(gt == 1)[0]
    print("tn {}, fp {}, fn {}, tp {} ----- f1-score {}".format(tn, fp, fn, tp, f1))

    i = 1
    pos, seq_tp, seq_fn, seq_fp = 0, 0, 0, 0

    while i < gt.shape[0]:
        if gt[i] == 1:
            pos += 1
            j = i
            while gt[i] == 1:
                i += 1

            if np.sum(est_cp[j:i]) > 0:
                seq_tp += 1
                est_cp[j:i] = 0
            else:
                seq_fn += 1

        i += 1

    seq_fp = np.where(np.diff(est_cp) == 1)[0].shape[0]
    seq_f1 = (2 * seq_tp) / (2 * seq_tp + seq_fn + seq_fp)

    print("SEQ : Pos {}, fp {}, fn {}, tp {} ----- f1-score {}".format(pos, seq_fp, seq_fn, seq_tp, seq_f1))
    result = "tn, {}, fp, {}, fn, {}, tp, {}, f1-score, {}, Pos, {}, seqfp, {}, seqfn, {}, seqtp, {}, seqf1, {}\n".format(tn, fp, fn, tp, f1, pos, seq_fp, seq_fn, seq_tp, seq_f1)
    return result

In [5]:
DS_NAME = 'USC'
DATA_PATH = './data/'
OUTPUT_PATH = os.path.join('./output/', DS_NAME)
MODEL_PATH = os.path.join('./output/', "model")
LOSS = 'nce'
SIM = 'cosine'
GPU = 0

WIN = 100
CODE_SIZE = 10
BATCH_SIZE = 32
EPOCHS = 40
LR = 1e-4
TEMP = 0.5
TAU = 0.1
BETA = 1
EVALFREQ = 25
decay_steps = 1000


train_name = "CP2_model_" + DS_NAME + "_T" + str(TEMP) + "_WIN" + str(WIN) + \
             "_BS" + str(BATCH_SIZE) + "_CS" + str(CODE_SIZE) + "_lr" + str(LR) + \
             "_LOSS" + LOSS +  "_SIM" + SIM + "_TAU" + str(TAU) + "_BETA" + str(BETA)
print("------------------------------------>>> " + train_name)

train_ds = load_dataset(DATA_PATH, DS_NAME, WIN, BATCH_SIZE, mode = "train")
test_ds = load_dataset(DATA_PATH, DS_NAME, WIN, BATCH_SIZE, mode = "test")


------------------------------------>>> CP2_model_USC_T0.5_WIN100_BS32_CS10_lr0.0001_LOSSnce_SIMcosine_TAU0.1_BETA1
number of samples : 4677 /  number of samples with change point : 175
train samples :  3741
(3741, 100) (3741, 1)
dataset shape :  (3741, 101)
number of samples : 4677 /  number of samples with change point : 175
test shape (936, 100) and number of change points 35 
(936, 100) (936, 1)
dataset shape :  (936, 101)


In [6]:
class ProjectionHead(nn.Module):
    def __init__(
        self, 
        blocks: List[Tuple[int, int, Optional[nn.Module], Optional[nn.Module]]]
    ):
        super(ProjectionHead, self).__init__()

        layers = []
        for input_dim, output_dim, batch_norm, non_linearity in blocks:
            use_bias = not bool(batch_norm)
            layers.append(nn.Linear(input_dim, output_dim, bias=use_bias))
            if batch_norm:
                layers.append(batch_norm)
            if non_linearity:
                layers.append(non_linearity)
        self.layers = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor):
        return self.layers(x)
        
class BarlowTwinsProjectionHead(ProjectionHead):
    def __init__(self,
                 input_dim: int = 2048,
                 hidden_dim: int = 8192,
                 output_dim: int = 8192):
        super(BarlowTwinsProjectionHead, self).__init__([
            (input_dim, hidden_dim, nn.BatchNorm1d(hidden_dim), nn.ReLU()),
            (hidden_dim, hidden_dim, nn.BatchNorm1d(hidden_dim), nn.ReLU()),
            (hidden_dim, output_dim, None, None),
        ])

In [7]:
class BarlowTwins(nn.Module):
    def __init__(self,
                 num_ftrs: int = 10,
                 proj_hidden_dim: int = 20,
                 out_dim: int = 40):

        super(BarlowTwins, self).__init__()

        self.num_ftrs = num_ftrs
        self.proj_hidden_dim = proj_hidden_dim
        self.out_dim = out_dim

        self.projection_mlp = BarlowTwinsProjectionHead(
            num_ftrs,
            proj_hidden_dim,
            out_dim
        )


    def forward(self,
                x0: torch.Tensor,
                x1: torch.Tensor = None,
                return_features: bool = False):
        
        f0 = x0.flatten(start_dim=1)
        out0 = self.projection_mlp(f0)

        if return_features:
            out0 = (out0, f0)

        if x1 is None:
            return out0

        f1 = x1.flatten(start_dim=1)
        out1 = self.projection_mlp(f1)

        if return_features:
            out1 = (out1, f1)

        return out0, out1

In [8]:
class BarlowTwinsLoss(torch.nn.Module):
    def __init__(
        self, 
        lambda_param: float = 5e-3, 
        gather_distributed : bool = False
    ):

        super(BarlowTwinsLoss, self).__init__()
        self.lambda_param = lambda_param
        self.gather_distributed = gather_distributed

    def forward(self, z_a: torch.Tensor, z_b: torch.Tensor) -> torch.Tensor:

        device = z_a.device

        z_a_norm = (z_a - z_a.mean(0)) / z_a.std(0)
        z_b_norm = (z_b - z_b.mean(0)) / z_b.std(0)

        N = z_a.size(0)
        D = z_a.size(1)

        c = torch.mm(z_a_norm.T, z_b_norm) / N 

        if self.gather_distributed and dist.is_initialized():
            world_size = dist.get_world_size()
            if world_size > 1:
                c = c / world_size
                dist.all_reduce(c)

        c_diff = (c - torch.eye(D, device=device)).pow(2)
        c_diff[~torch.eye(D, dtype=bool)] *= self.lambda_param
        loss = c_diff.sum()

        return loss

In [22]:
class ResidualBlock(nn.Module):
    
    def __init__(self,
                 in_channels,
                 dilation_rate: int,
                 nb_filters: int,
                 kernel_size: int,
                 dropout_rate: float = 0, 
                 use_batch_norm: bool = False,
                 use_layer_norm: bool = False,
                 use_weight_norm: bool = False, 
                 training: bool = True):
        """Defines the residual block for the WaveNet TCN
        Args:
            dilation_rate: The dilation power of 2 we are using for this residual block
            nb_filters: The number of convolutional filters to use in this block
            kernel_size: The size of the convolutional kernel
            activation: The final activation used in o = Activation(x + F(x))
            dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
        """

        self.dilation_rate = dilation_rate
        self.nb_filters = nb_filters
        self.kernel_size = kernel_size
        self.dropout_rate = dropout_rate
        # for causal padding
        self.padding = (self.kernel_size - 1) * self.dilation_rate
        
        self.use_batch_norm = use_batch_norm
        self.use_layer_norm = use_layer_norm
        self.use_weight_norm = use_weight_norm
        
        self.training = training

        super(ResidualBlock, self).__init__()
        
        self.conv_1 = nn.Conv1d(in_channels, self.nb_filters, self.kernel_size, 
                                padding=0, dilation=self.dilation_rate)        
        if self.use_weight_norm:
            weight_norm(self.conv_1) 
        self.bn_1 = nn.BatchNorm1d(self.nb_filters)
        self.ln_1 = nn.LayerNorm(self.nb_filters)              
        self.relu_1 = nn.ReLU()

        self.conv_2 = nn.Conv1d(self.nb_filters, self.nb_filters, self.kernel_size, 
                                padding=0, dilation=self.dilation_rate)        
        if self.use_weight_norm:
            weight_norm(self.conv_1)    
        self.bn_2 = nn.BatchNorm1d(self.nb_filters)
        self.ln_2 = nn.LayerNorm(self.nb_filters)              
        self.relu_2 = nn.ReLU()        
        
        self.conv_block = nn.Sequential()
        self.downsample = nn.Conv1d(in_channels, self.nb_filters, kernel_size=1) if in_channels != self.nb_filters else nn.Identity()
        
        self.relu = nn.ReLU()  
                
        self.init_weights()
        
        
    def init_weights(self):
        # in the realization, they use random normal initialization
        torch.nn.init.normal_(self.conv_1.weight, mean=0, std=0.05)
        torch.nn.init.zeros_(self.conv_1.bias)            
        
        torch.nn.init.normal_(self.conv_2.weight, mean=0, std=0.05)
        torch.nn.init.zeros_(self.conv_2.bias)            
        
        if isinstance(self.downsample, nn.Conv1d):         
            torch.nn.init.normal_(self.downsample.weight, mean=0, std=0.05)
            torch.nn.init.zeros_(self.downsample.bias)                    
            
    def forward(self, inp):
        # inp batch, channels, time
        ######################
        # do causal padding        
        out = F.pad(inp, (self.padding, 0))
        out = self.conv_1(out)
        
        if self.use_batch_norm:
            out = self.bn_1(out)
        elif self.use_layer_norm:
            out = self.ln_1(out)        
        out = self.relu_1(out)
        
        # spatial dropout
        out = out.permute(0, 2, 1)   # convert to [batch, time, channels]
        out = F.dropout2d(out, self.dropout_rate, training=self.training)        
        out = out.permute(0, 2, 1)   # back to [batch, channels, time]    
        
        #######################
        # do causal padding
        out = F.pad(out, (self.padding, 0))
        out = self.conv_2(out)
        if self.use_batch_norm:
            out = self.bn_2(out)
        elif self.use_layer_norm:
            out = self.ln_2(out)
        out = self.relu_2(out)            
        out = self.relu_2(out)    
        # spatial dropout
        # out batch, channels, time 
        
        out = out.permute(0, 2, 1)   # convert to [batch, time, channels]
        out = F.dropout2d(out, self.dropout_rate, training=self.training)
        out = out.permute(0, 2, 1)   # back to [batch, channels, time]            
        
        #######################        
        skip_out = self.downsample(inp)
        #######################
        res = self.relu(out + skip_out)
        return res, skip_out
    
# only causal padding
# only return sequence = True
    
class TCN(nn.Module):        
    def __init__(self,
                 in_channels=1,
                 nb_filters=64,
                 kernel_size=3,
                 nb_stacks=1,
                 dilations=(1, 2, 4, 8, 16, 32),
                 use_skip_connections=True,
                 dropout_rate=0.0, 
                 use_batch_norm: bool = False,
                 use_layer_norm: bool = False, 
                 use_weight_norm: bool = False):

        super(TCN, self).__init__()
        
        self.dropout_rate = dropout_rate
        self.use_skip_connections = use_skip_connections
        self.dilations = dilations
        self.nb_stacks = nb_stacks
        self.kernel_size = kernel_size
        self.nb_filters = nb_filters
        
        self.use_batch_norm = use_batch_norm
        self.use_layer_norm = use_layer_norm
        self.use_weight_norm = use_weight_norm
        self.in_channels = in_channels
        if self.use_batch_norm + self.use_layer_norm + self.use_weight_norm > 1:
            raise ValueError('Only one normalization can be specified at once.')        
        
        self.residual_blocks = []        
        res_block_filters = 0
        for s in range(self.nb_stacks):
            for i, d in enumerate(self.dilations):
                in_channels = self.in_channels if i + s == 0 else res_block_filters                
                res_block_filters = self.nb_filters[i] if isinstance(self.nb_filters, list) else self.nb_filters
                self.residual_blocks.append(ResidualBlock(in_channels=in_channels, 
                                                          dilation_rate=d,
                                                          nb_filters=res_block_filters,
                                                          kernel_size=self.kernel_size,
                                                          dropout_rate=self.dropout_rate, 
                                                          use_batch_norm=self.use_batch_norm,
                                                          use_layer_norm=self.use_layer_norm,
                                                          use_weight_norm=self.use_weight_norm))

        
        self.residual_blocks = nn.ModuleList(self.residual_blocks)
                                            
    def forward(self, inp):
        out = inp
        for layer in self.residual_blocks:
            out, skip_out = layer(out)
        if self.use_skip_connections:
            out = out + skip_out
        return out

########################### model #########################################
class Encoder(nn.Module):
    def __init__(self, c_in=1, nb_filters=64, kernel_size=4, 
                 dilations=[1,2,4,8], nb_stacks=2, n_steps=50, code_size=10, seq_len=100):       
        super(Encoder, self).__init__()        
        
        self.tcn_layer = TCN(in_channels=c_in, nb_filters=nb_filters, 
                             nb_stacks=nb_stacks, dilations=dilations, use_skip_connections=True, dropout_rate=0)
        
        self.fc1 = nn.Linear(nb_filters * seq_len, 2 * n_steps)  
        self.fc2 = nn.Linear(2 * n_steps, n_steps)    
        self.output_layer = nn.Linear(n_steps, code_size)           
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = x
        if len(out.shape) == 2:
            out = out.unsqueeze(1)
        out = self.tcn_layer(out)     
        out = out.flatten(1, 2)         
        out = self.relu(self.fc1(out)) 
        out = self.relu(self.fc2(out)) 
        out = self.output_layer(out)
        return out
    
########################### loss #########################################
def _cosine_simililarity_dim2(x, y):
    cos = nn.CosineSimilarity(dim=2, eps=1e-6)
    v = cos(x.unsqueeze(1), y.unsqueeze(0))
    return v    


################## PL wrapper ###############################################
class BTSCP_model(pl.LightningModule):
    def __init__(
        self,
        encoder: nn.Module,
        model: nn.Module,     
        train_dataset: Dataset, 
        test_dataset: Dataset, 
        batch_size: int = 64,        
        num_workers: int = 2,        
        temperature: float = 0.1, 
        lr: float = 1e-4,
        decay_steps: int = 1000, 
        window_1: int = 100,
        window_2: int = 100
    ) -> None:
        super().__init__()
        self.encoder = encoder
        self.model = model
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        
        self.batch_size = batch_size
        self.num_workers = num_workers        
        
        self.temperature = temperature
        
        self.lr = lr
        self.decay_steps = decay_steps   
        
        self.window = window_1
        self.window_1 = window_1
        self.window_2 = window_2
        self.loss = BarlowTwinsLoss()

    def forward(self, inputs1: torch.Tensor, inputs2=None) -> torch.Tensor:
        inp1 = self.encoder(inputs1)
        inp2 = self.encoder(inputs2)
        return self.model(inp1, inp2)
    
    def pred(self, inputs):
        return self.encoder(inputs)
    
    def training_step(self, batch1, batch2):
       
        emb1, emb2 = self.forward(batch1.float(), batch2.float())

        loss = self.loss.forward(emb1, emb2)

        return loss
        
    def validation_step(self, batch1, batch2):

        emb1, emb2 = self.forward(batch1.float(), batch2.float())

        loss = self.loss.forward(emb1, emb2)

        return loss


    def configure_optimizers(self) -> torch.optim.Optimizer:
        opt = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        return opt

    def train_dataloader(self) -> DataLoader:
        return DataLoader(
            self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers
        )

    def val_dataloader(self) -> DataLoader:
        return DataLoader(
            self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers
        )
    def get_enc(self):
        return self.encoder
    
def _cosine_simililarity_dim1(x, y):
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    v = cos(x, y)
    return v   

In [13]:
def random_crop(batch, WIN):
    tmp = batch[0][:,1:]
    l = torch.randint(0, WIN+1, (1,))
    l = max(l, int(0.8 * tmp.shape[1]))
    pos = torch.randint(0, WIN // 10, (1,))
    tmp = tmp[:, pos:l+1]
    tmp = F.pad(input=tmp, pad=(WIN - tmp.shape[1], 0), mode='constant', value=0)
    return tmp

In [14]:
def random_noise(batch, sigma=1):
    tmp = batch[0][:,1:]
    noise = torch.normal(0, sigma, size=tmp.shape)
    tmp += noise
    return tmp

In [15]:
enc = Encoder(code_size = CODE_SIZE, seq_len = WIN)
model = BarlowTwins(CODE_SIZE, 2*CODE_SIZE, 4*CODE_SIZE)

model = BTSCP_model(enc, model, train_ds, test_ds, batch_size=BATCH_SIZE, temperature=TEMP, lr=LR, decay_steps=decay_steps, window_1=WIN)
optimizer = model.configure_optimizers()

In [13]:
train_loader = model.train_dataloader()
val_loader = model.val_dataloader()

train_losses = []
val_losses = []

for epoch in tqdm(range(EPOCHS)):
    
    iteration = 0
    train_losses_iters = []

    
    for index, batch in enumerate(train_loader):
        aug1, aug2 = random_crop(batch, WIN), random_crop(batch, WIN)
        loss = model.training_step(aug1, aug2)
        train_losses_iters.append(float(loss))

        print("train_losses_iters", train_losses_iters[-1])

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        iteration += 1

        if not iteration % 50:
            model.eval()
            vall = []
            with torch.no_grad():
                for c, b in enumerate(val_loader):
                    aug1, aug2 = random_crop([b], WIN), random_crop([b], WIN)
                    val_loss = model.validation_step(aug1, aug2)
                    vall.append(float(val_loss.detach()))
                    if c>10:
                        break
                print("val_loss", np.mean(vall))
                val_losses.append(np.mean(vall))
          
            
            model.train()
        
    print("epoch_train_loss", np.mean(train_losses_iters))  
    train_losses.append(np.mean(train_losses_iters))
    if len(val_losses):
        print("epoch_val_loss", np.mean(val_losses))


/home/bogdan/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 2 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
  0%|                                                    | 0/40 [00:00<?, ?it/s]/home/bogdan/.local/lib/python3.10/site-packages/torch/nn/functional.py:1338: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dropout beh

train_losses_iters 24.186826705932617
train_losses_iters 2.035172462463379
train_losses_iters 1.9536380767822266
train_losses_iters 25.047222137451172
train_losses_iters 2.090066909790039
train_losses_iters 2.0098931789398193
train_losses_iters 2.02724552154541
train_losses_iters 1.8167611360549927
train_losses_iters 2.1229002475738525
train_losses_iters 34.95899200439453
train_losses_iters 2.5307252407073975
train_losses_iters 27.999502182006836
train_losses_iters 12.878658294677734
train_losses_iters 2.2299301624298096
train_losses_iters 2.751626968383789
train_losses_iters 26.711463928222656
train_losses_iters 2.1358089447021484
train_losses_iters 2.435905933380127
train_losses_iters 1.854870319366455
train_losses_iters 3.178393602371216
train_losses_iters 3.9356865882873535
train_losses_iters 12.22683048248291
train_losses_iters 23.78879165649414
train_losses_iters 2.4218716621398926
train_losses_iters 1.478054165840149
train_losses_iters 3.116790294647217
train_losses_iters 1.7452

  2%|█                                           | 1/40 [00:42<27:51, 42.86s/it]

epoch_train_loss 8.545247483457256
epoch_val_loss 19.280677288770676
train_losses_iters 2.06050968170166
train_losses_iters 2.8866240978240967
train_losses_iters 30.311492919921875
train_losses_iters 2.037013530731201
train_losses_iters 2.2524216175079346
train_losses_iters 1.470069169998169
train_losses_iters 1.9341477155685425
train_losses_iters 2.325690269470215
train_losses_iters 2.1072514057159424
train_losses_iters 27.172317504882812
train_losses_iters 24.776836395263672
train_losses_iters 1.9360636472702026
train_losses_iters 21.571063995361328
train_losses_iters 2.3894832134246826
train_losses_iters 2.887561559677124
train_losses_iters 22.764022827148438
train_losses_iters 2.347937822341919
train_losses_iters 2.4910688400268555
train_losses_iters 21.184417724609375
train_losses_iters 1.5194916725158691
train_losses_iters 5.592536449432373
train_losses_iters 2.3087263107299805
train_losses_iters 4.802385330200195
train_losses_iters 3.462223529815674
train_losses_iters 18.9609928

  5%|██▏                                         | 2/40 [01:24<26:50, 42.37s/it]

...

train_losses_iters 2.5502350330352783
epoch_train_loss 2.9370141905597134
epoch_val_loss 14.955759647776162
train_losses_iters 2.9482674598693848
train_losses_iters 4.0173563957214355
train_losses_iters 6.155426979064941
train_losses_iters 2.469275951385498
train_losses_iters 2.3316149711608887
train_losses_iters 1.9488478899002075
train_losses_iters 2.4041671752929688
train_losses_iters 2.2452688217163086
train_losses_iters 2.518098831176758
train_losses_iters 4.132260799407959
train_losses_iters 2.2657222747802734
train_losses_iters 2.208374261856079
train_losses_iters 9.115715026855469
train_losses_iters 2.128159761428833
train_losses_iters 3.2734365463256836
train_losses_iters 4.161949157714844
train_losses_iters 3.942803382873535
train_losses_iters 2.192927837371826
train_losses_iters 2.598845958709717
train_losses_iters 2.3755955696105957
train_losses_iters 3.8021652698516846
train_losses_iters 2.5035274028778076
train_losses_iters 2.358328342437744
train_losses_iters 2.044609785

 98%|█████████████████████████████████████████▉ | 39/40 [19:53<00:28, 28.15s/it]

epoch_train_loss 3.1527431867061515
epoch_val_loss 14.971449659548256
train_losses_iters 2.2827587127685547
train_losses_iters 2.6508185863494873
train_losses_iters 2.4359335899353027
train_losses_iters 2.13018798828125
train_losses_iters 2.2107620239257812
train_losses_iters 2.2299208641052246
train_losses_iters 4.161649703979492
train_losses_iters 2.02751088142395
train_losses_iters 2.198331832885742
train_losses_iters 2.886848211288452
train_losses_iters 4.163015365600586
train_losses_iters 5.631760597229004
train_losses_iters 2.3748879432678223
train_losses_iters 2.444596290588379
train_losses_iters 2.3881640434265137
train_losses_iters 2.491826057434082
train_losses_iters 2.633986473083496
train_losses_iters 2.412400960922241
train_losses_iters 2.2940735816955566
train_losses_iters 2.105245590209961
train_losses_iters 3.462643623352051
train_losses_iters 2.0677101612091064
train_losses_iters 7.6075029373168945
train_losses_iters 2.0583858489990234
train_losses_iters 2.248976945877

100%|███████████████████████████████████████████| 40/40 [20:20<00:00, 30.52s/it]

train_losses_iters 5.358888149261475
epoch_train_loss 2.9911909867555666
epoch_val_loss 14.87304739529888


In [15]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, './barlowtscp'+DS_NAME+str(WIN)+str(CODE_SIZE)+str(BATCH_SIZE)+'_opt.pt')

In [16]:
cpt = torch.load('barlowtscpUSC1001032_opt.pt')
model.load_state_dict(cpt['model_state_dict'])

<All keys matched successfully>

In [17]:
enc = model.get_enc()

In [19]:
test_ds = load_future(DATA_PATH, DS_NAME, WIN, BATCH_SIZE, mode = "test")

number of samples : 2336 /  number of samples with change point : 175
test shape (468, 200) and number of change points 35 
(468, 200) (468, 1)
dataset shape :  (468, 201)


In [30]:
x_test, lbl_test = test_ds[:,1:], test_ds[:,0]

num = x_test.shape[0]
lbl_test = np.array(lbl_test).reshape((lbl_test.shape[0], 1))
history = enc(torch.from_numpy(x_test[:, 0:WIN].reshape((num, 1, WIN))).float())
future = enc(torch.from_numpy(x_test[:, WIN:].reshape((num, 1, WIN))).float())
pred_out = np.concatenate((lbl_test, history.detach().numpy(), future.detach().numpy()), 1)
rep_sim = _cosine_simililarity_dim1(history, future)

print('Average similarity for test set : Reps : {}'.format(np.mean(rep_sim.detach().numpy())))
gt = np.zeros(lbl_test.shape[0])
gt[np.where((lbl_test > int(2 * WIN * 0.15)) & (lbl_test < int(2 * WIN * 0.85)))[0]] = 1
result = estimate_CPs(rep_sim.detach().numpy(), gt, os.path.join(OUTPUT_PATH, train_name),
                    os.path.join(OUTPUT_PATH, "Evaluation.txt"),
                    metric='cosine', threshold=0.5)

/home/bogdan/.local/lib/python3.10/site-packages/torch/nn/functional.py:1338: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dropout behavior, please switch to using dropout1d instead.
  warnings.warn("dropout2d: Received a 3D input to dropout2d and assuming that channel-wise "


Average similarity for test set : Reps : 0.999922513961792
tn 442, fp 0, fn 26, tp 0 ----- f1-score 0.0
SEQ : Pos 7, fp 0, fn 7, tp 0 ----- f1-score 0.0


In [36]:
result = estimate_CPs(rep_sim.detach().numpy(), gt, os.path.join(OUTPUT_PATH, train_name),
                    os.path.join(OUTPUT_PATH, "Evaluation.txt"),
                    metric='cosine', threshold=0.9)

tn 442, fp 0, fn 26, tp 0 ----- f1-score 0.0
SEQ : Pos 7, fp 0, fn 7, tp 0 ----- f1-score 0.0
